# Decomposition Tutorial

This tutorial will show you how to load a geometry from an STL file, create a grid object using this geometry, and then to decompose the grid into load-balanced domains.

First, load a geometry and visualize it.

In [4]:
from slagg import Geometry

geometry = Geometry("../tests/stl_files/Moon.stl")
geom.plot()

ModuleNotFoundError: No module named 'slagg'

Next, we create a grid to discretize this geometry.  The grid is made up of cells, some of which lie inside the geometry and others are outside.  We assume that the desired simulation domain is inside the object.  Providing the Grid with a Geometry object will calculate intersections and mark all cells interior to the object.

In [7]:
from slagg import Grid
grid = Grid((30, 8, 60), geometry=geometry)
grid.plot()

ModuleNotFoundError: No module named 'slagg'

The above plot shows only the cells that contain geometry.

Now, we want to generate a decomposition for this grid.  The standard, default decomposition generates regions that are as cubic and equal in volume as possible, based on the number of regions desired and the number of cells in the grid.  With complex geometries, this is often not a good, load-balanced decomposition.

In [8]:
N_desired_domains = 32
decomp = Decomp(grid, N_desired_domains, geometry_biased=False)
ax = geom.plot(plot=False)
decomp.plot(axes=ax)

NameError: name 'Decomp' is not defined

The above decomposition does not take into accout any geometries, but just seeks to make domains with a balanced number of cells.  If instead, we seek to balance the number of geometry-containing cells, a better decomposition results:

In [ ]:
decomp = Decomp(grid, N_desired_domains, geometry_biased=True)
ax = geom.plot(plot=False)
decomp.plot(axes=ax)